In [1]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_groq import ChatGroq

# Load environment variables from .env
print(load_dotenv())


True


In [2]:
# Create a ChatGroq model
model = ChatGroq(model="llama-3.1-8b-instant")

# Define prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert product reviewer."),
        ("human", "List the main features of the product {product_name}."),
    ]
)


# Define pros analysis step
def analyze_pros(features):
    pros_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert product reviewer."),
            (
                "human",
                "Given these features: {features}, list the pros of these features.",
            ),
        ]
    )
    return pros_template.format_prompt(features=features)


# Define cons analysis step
def analyze_cons(features):
    cons_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert product reviewer."),
            (
                "human",
                "Given these features: {features}, list the cons of these features.",
            ),
        ]
    )
    return cons_template.format_prompt(features=features)


# Combine pros and cons into a final review
def combine_pros_cons(pros, cons):
    return f"Pros:\n{pros}\n\nCons:\n{cons}"


In [5]:
list_features_chain = prompt_template |model | StrOutputParser()

features = list_features_chain.invoke({"product_name": "ipods"})

print(features)

iPods are a line of portable media players developed by Apple Inc. Here are the main features of iPods:

1. **Storage Capacity**: iPods come in various storage sizes, ranging from 32 GB to 1 TB, allowing users to store thousands of songs, photos, and videos.

2. **Touchscreen Display**: Many iPod models feature a touchscreen display, making it easy to navigate and control the device with the touch of a finger.

3. **Music and Video Playback**: iPods support various audio and video formats, including MP3, AAC, and H.264. Users can play music, watch videos, and listen to podcasts on the go.

4. **Connectivity Options**: iPods often feature connectivity options like Wi-Fi, Bluetooth, and USB, allowing users to sync their device with their computer and connect to other devices wirelessly.

5. **Battery Life**: iPods have a long battery life, with some models lasting up to 40 hours on a single charge.

6. **Water Resistance**: Some iPod models, like the iPod touch, offer water resistance, m

In [ ]:
# Simplify branches with LCEL
pros_branch_chain = (
    RunnableLambda(lambda x: analyze_pros(x)) | model | StrOutputParser()
)

cons_branch_chain = (
    RunnableLambda(lambda x: analyze_cons(x)) | model | StrOutputParser()
)

In [ ]:
cons = cons_branch_chain.invoke(features)

While the features of iPods are impressive, there are some potential drawbacks to consider:

**1. Storage Capacity:**
- **Cost**: The higher storage capacities can be expensive, making it less accessible to budget-conscious consumers.
- **Data Overload**: With the ability to store thousands of songs, photos, and videos, users may struggle to manage and organize their content, leading to clutter and disorganization.
- **Battery Drain**: Larger storage capacities can cause battery drain, as the device needs to work harder to manage and access the stored data.

**2. Touchscreen Display:**
- **Fingerprint Smudges**: Touchscreens can be prone to fingerprint smudges and smears, which can be distracting and frustrating.
- **Screen Damage**: Touchscreens are more susceptible to damage from drops or scratches, which can be costly to repair or replace.
- **Eye Strain**: Staring at a screen for extended periods can cause eye strain and fatigue.

**3. Music and Video Playback:**
- **Format Limitat

In [13]:
# Create the combined chain using LangChain Expression Language (LCEL)
incomplete_chain = (
    list_features_chain
    | RunnableParallel(branches={"pros": pros_branch_chain, "cons": cons_branch_chain})
)

complete_chain = (
    list_features_chain
    | RunnableParallel(branches={"pros": pros_branch_chain, "cons": cons_branch_chain})
    | RunnableLambda(lambda x: f'Pros:\n{x["branches"]["pros"]}\n\nCons:\n{x["branches"]["cons"]}')
)

# Run the chain
result = complete_chain.invoke({"product_name": "MacBook Pro"})

# Output
print(result)

Pros:
Based on the features listed, here are the pros of the MacBook Pro:

1. **Display**:
   - High-resolution Retina display for vibrant colors and crisp text.
   - Offers a great viewing experience for watching videos, browsing the internet, and working on creative projects.

2. **Processor**:
   - Fast performance with Apple's M2, M2 Pro, or M2 Max chip.
   - Efficient battery life, allowing users to work or play for extended periods without needing to recharge.

3. **Memory and Storage**:
   - Up to 96 GB of RAM for smooth multitasking and running multiple applications simultaneously.
   - Up to 8 TB of storage, providing ample space for storing files, videos, and photos.

4. **Graphics**:
   - Dedicated graphics card for improved graphics performance for gaming, video editing, and other resource-intensive tasks.
   - Supports 16-core or 32-core GPU, making it suitable for demanding tasks.

5. **Battery Life**:
   - Up to 22 hours of battery life, allowing users to work or play fo

In [11]:
import json

print(json.dumps(result, indent=2))
print(type(result))

{
  "branches": {
    "pros": "Based on the provided features of the MacBook Pro, here are the pros of each feature:\n\n**General Features**\n\n1. **Design**: Sleek and lightweight design makes it easy to carry around, and the Retina display provides a visually appealing experience.\n2. **Operating System**: macOS operating system offers a seamless user experience, with intuitive interfaces and robust security features.\n3. **Portability**: The lightweight and compact design of the MacBook Pro makes it an ideal choice for professionals and students who need to work on-the-go.\n\n**Display Features**\n\n1. **Retina Display**: The high-resolution display provides a crisp and clear visual experience, making it ideal for tasks such as video editing, graphic design, and gaming.\n2. **Color Accuracy**: The wide color gamut and P3 color space ensure accurate color representation, making it a great choice for graphic designers, photographers, and videographers.\n3. **Brightness**: The up to 10